<a href="https://colab.research.google.com/github/gitan5hi/Bias-Detection-in-COMPAS-dataset/blob/main/Bias_Detection_in_COMPAS_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading dataset

In [1]:
!pip install kaggle #in the terminal

In [2]:
##Uploading Kaggle API token
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"gitanshisingh","key":"f1f70925013e0f03eef4fb75e0a7cc15"}'}

In [3]:
##Set up the Kaggle API credentials for use in the colab environment
!mkdir -p ~/.kaggle   #creates a hidden directory for storing Kaggle credentials
!mv kaggle.json ~/.kaggle/   #moves the downloaded API token to this secure location
!chmod 600 ~/.kaggle/kaggle.json   #only the owner can access the file, protectin sensitive information

In [4]:
##Downloading the dataset
!kaggle datasets download -d danofer/compass

Dataset URL: https://www.kaggle.com/datasets/danofer/compass
License(s): DbCL-1.0
  0% 0.00/2.72M [00:00<?, ?B/s]
100% 2.72M/2.72M [00:00<00:00, 207MB/s]


In [5]:
!unzip compass.zip -d compas_dataset

Archive:  compass.zip
  inflating: compas_dataset/compas-scores-raw.csv  
  inflating: compas_dataset/cox-violent-parsed.csv  
  inflating: compas_dataset/cox-violent-parsed_filt.csv  
  inflating: compas_dataset/propublicaCompassRecividism_data_fairml.csv/._propublica_data_for_fairml.csv  
  inflating: compas_dataset/propublicaCompassRecividism_data_fairml.csv/propublica_data_for_fairml.csv  


In [6]:
import pandas as pd
df=pd.read_csv("compas_dataset/propublicaCompassRecividism_data_fairml.csv/propublica_data_for_fairml.csv")
print(df.head())

   Two_yr_Recidivism  Number_of_Priors  score_factor  Age_Above_FourtyFive  \
0                  0                 0             0                     1   
1                  1                 0             0                     0   
2                  1                 4             0                     0   
3                  0                 0             0                     0   
4                  1                14             1                     0   

   Age_Below_TwentyFive  African_American  Asian  Hispanic  Native_American  \
0                     0                 0      0         0                0   
1                     0                 1      0         0                0   
2                     1                 1      0         0                0   
3                     0                 0      0         0                0   
4                     0                 0      0         0                0   

   Other  Female  Misdemeanor  
0      1       0        

**Attributes Explained**

* *Two_yr_Recidivism*: The target variable; indicates if a defendant was rearrested (recidivated) within two years (1=yes, 0=no).

* *Number_of_Priors*: Count of prior offenses before the most recent arrest for each defendant.

* *score_factor*: The COMPAS system assigns risk scores to defendants predicting their likelihood of recidivism (reoffending) within a certain period (e.g., two years) based on various behavioral, demographic, and criminal history factors.

* *Age_Above_FourtyFive, Age_Below_TwentyFive*: Binary features showing if age is above 45 or below 25, respectively, encoding age group buckets for fairness analysis.

* *African_American, Asian, Hispanic, Native_American, Other*: One-hot encoded columns for racial groups. These are mutually exclusive, so a single '1' per row indicates racial membership.

* *Female*: Binary feature for gender.

* *Misdemeanor*: Binary indicator for the type of charge (misdemeanor vs. felony).




# Project Outcome

This analysis checks if the model that predicts whether someone might commit another crime is unfair to certain groups, like people of different  races, ages, or genders.


The model looks at details such as how many crimes a person did before, their age, race, and if their current charge is serious or not.


If a model is biased, it could mean certain people get harsher treatement from the justics system, not because of facts, but because of unfair preictions made by a computer.

Bias detection helps ensure everyone gets equal treatment, and that the model's decisions are just and reliable for all backgrounds.


# Data preprocessing

In [7]:
# import libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from collections import Counter

In [8]:
# Normalization
scaler = MinMaxScaler()
df['Number_of_Priors_Normalized'] = scaler.fit_transform(df[['Number_of_Priors']])

In [9]:
## Look for correlation of Two_yr_Recidivism with other attributes
print(df.info())
correlation_matrix=df.corr()
print(correlation_matrix["Two_yr_Recidivism"].sort_values(ascending=False))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6172 entries, 0 to 6171
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Two_yr_Recidivism            6172 non-null   int64  
 1   Number_of_Priors             6172 non-null   int64  
 2   score_factor                 6172 non-null   int64  
 3   Age_Above_FourtyFive         6172 non-null   int64  
 4   Age_Below_TwentyFive         6172 non-null   int64  
 5   African_American             6172 non-null   int64  
 6   Asian                        6172 non-null   int64  
 7   Hispanic                     6172 non-null   int64  
 8   Native_American              6172 non-null   int64  
 9   Other                        6172 non-null   int64  
 10  Female                       6172 non-null   int64  
 11  Misdemeanor                  6172 non-null   int64  
 12  Number_of_Priors_Normalized  6172 non-null   float64
dtypes: float64(1), int

In [10]:
# Reweighing based on 'African_American' group
group_counts = df['African_American'].value_counts().to_dict()
total_samples = len(df)
weights = {grp: total_samples/(len(group_counts)*count) for grp, count in group_counts.items()}
sample_weights = df['African_American'].map(weights)

In [11]:
# Sampling: oversample minority African_American group to balance dataset
df_majority = df[df['African_American'] == 0]
df_minority = df[df['African_American'] == 1]
df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=42)
df_balanced = pd.concat([df_majority, df_minority_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

In [12]:
# Feature transformation
features_to_use = ['Number_of_Priors', 'score_factor', 'Age_Above_FourtyFive', 'Age_Below_TwentyFive', 'Misdemeanor', 'Number_of_Priors_Normalized']
df_balanced_features = df_balanced[features_to_use]
df_balanced_target = df_balanced['Two_yr_Recidivism']

In [13]:
##Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_balanced_features, df_balanced_target, test_size=0.2, random_state=42)
print(f"Before balancing: {Counter(df['African_American'])}")
print(f"After balancing: {Counter(df_balanced['African_American'])}")

Before balancing: Counter({1: 3175, 0: 2997})
After balancing: Counter({0: 2997, 1: 2997})


# Model Development and Training

Different *Predictive Models* for binary outcome (Logistic regression, Decision Tree) using features from the dataset.

In [14]:
##Train logistic regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logreg=LogisticRegression(max_iter=100)
logreg.fit(X_train, y_train)
#Predict and evaluate accuracy
logreg_preds=logreg.predict(X_test)
print("Logistic Regression Accuracy:", accuracy_score(y_test, logreg_preds))

Logistic Regression Accuracy: 0.6872393661384487


Linear Regression (a Predictive ML model) is not used here because the outcome is binary and it predicts continuous values.

In [15]:
##Train Random Forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf=RandomForestClassifier(n_estimators=10, random_state=42)
rf.fit(X_train,y_train)
rf_preds=rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, rf_preds))

Random Forest Accuracy: 0.6930775646371977


# Bias Detection and Measurement

In [17]:
import numpy as np
from sklearn.metrics import confusion_matrix

# For bias metrics, put sensitive attributes aside for group-wise evaluation
# List of individual minority columns
minority_columns = ['African_American', 'Asian', 'Hispanic', 'Native_American', 'Other','Female']

#Sums values across minority columns for each row to check if a person belongs to any minority group (sum > 0).
#Converts this boolean result into binary integers (1 if minority; 0 otherwise), creating a new series any_minority indicating minority status for group-wise bias evaluation.
#any_minority = (df.loc[y_test.index, minority_columns].sum(axis=1) > 0).astype(int)

def disparity_impact(y_true, y_pred, sensitive_attr):
    # Ratio of favorable outcomes for protected vs unprotected group
    favorable_protected = np.mean(y_pred[sensitive_attr == 1] == 0)
    favorable_unprotected = np.mean(y_pred[sensitive_attr == 0] == 0)
    return favorable_protected / favorable_unprotected

def statistical_parity_difference(y_pred, sensitive_attr):
    p_privileged = np.mean(y_pred[sensitive_attr == 0] == 0)
    p_protected = np.mean(y_pred[sensitive_attr == 1] == 0)
    return p_protected - p_privileged

def false_positive_rate_difference(y_true, y_pred, sensitive_attr):
    cm_protected = confusion_matrix(y_true[sensitive_attr == 1], y_pred[sensitive_attr == 1])
    cm_unprotected = confusion_matrix(y_true[sensitive_attr == 0], y_pred[sensitive_attr == 0])

    def safe_fpr(cm):
      if cm.shape == (2,2):
        tn,fp=cm[0,0],cm[0,1]
        if (fp+tn)==0:
          return 0
        else:
          return fp/(fp+tn)
      else:
        return 0
    fpr_protected = safe_fpr(cm_protected)
    fpr_unprotected = safe_fpr(cm_unprotected)
    return fpr_protected - fpr_unprotected

#Dictionary to store fairness metrics for each minority group
fairness_metrics={}

# Interate over each minority group column to calculate metrics separately
for group in minority_columns:
  sensitive_attr = df.loc[y_test.index, group]

  di = disparity_impact(y_test, logreg_preds, sensitive_attr)
  spd = statistical_parity_difference(logreg_preds, sensitive_attr)
  fprd = false_positive_rate_difference(y_test, logreg_preds, sensitive_attr)

  fairness_metrics[group] = {
        'Disparate_Impact': di,
        'Statistical_Parity_Difference': spd,
        'False_Positive_Rate_Difference': fprd
  }

# Print the fairness metrics grouped by minority category
for group, metrics in fairness_metrics.items():
    print(f"Metrics for {group}:")
    print(f"  Disparate Impact: {metrics['Disparate_Impact']:.3f}")
    print(f"  Statistical Parity Difference: {metrics['Statistical_Parity_Difference']:.3f}")
    print(f"  False Positive Rate Difference: {metrics['False_Positive_Rate_Difference']:.3f}\n")


Metrics for African_American:
  Disparate Impact: 0.984
  Statistical Parity Difference: -0.009
  False Positive Rate Difference: 0.021

Metrics for Asian:
  Disparate Impact: 1.138
  Statistical Parity Difference: 0.081
  False Positive Rate Difference: -0.080

Metrics for Hispanic:
  Disparate Impact: 1.026
  Statistical Parity Difference: 0.015
  False Positive Rate Difference: -0.021

Metrics for Native_American:
  Disparate Impact: 1.708
  Statistical Parity Difference: 0.414
  False Positive Rate Difference: -0.247

Metrics for Other:
  Disparate Impact: 0.821
  Statistical Parity Difference: -0.106
  False Positive Rate Difference: 0.046

Metrics for Female:
  Disparate Impact: 1.071
  Statistical Parity Difference: 0.041
  False Positive Rate Difference: -0.065



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
